In [1]:
import glob
import os
import re
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import spacy
import textacy

In [13]:
test = os.path.join("plain", "36105213330389.txt")

In [ ]:
nlp = spacy.load('en')

In [14]:
nlplg = spacy.load('en_core_web_lg')

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/csb5t/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [44]:
with open(test, 'r') as f:
    text = f.read()
    cleaned = clean_text(text[:5000])
    print(cleaned)
    

the white hat imniiiniimium unmm arrowsmith and síxpe ny seríesarrow series men in boat to say nothing of the dog the end of life yol diary of pilgrimage iii recalled to life iy frenchman in america and the supernatural the white hat vol vii quay street slmpkin hamilton kent cracker over the copyright entered at stationers hall the white hat and other stories by author of sporting recollections flowers of the hunt etc juustrations bg tbe îlutbor ii quay street london simpkin kent limitednotice the following stories viz the vicissitudes of how miss went to the little fairy and got at have already appeared in the columns of the illustrated sporting and dramatic news to the courtesy of whose proprietor the author is indebted for per mission to republish them contents rage the white hat vicissitudes of hack how miss brown went to the derby little fairy house in the country got at the queen of the arena uncle new way to pay old debts the black sheep of the family game to the last the bishop

In [30]:
with open(test, 'r') as f:
    text = f.read()
    doc = nlplg(text)

In [40]:
word = doc[199]
word.is_alpha

True

In [55]:
def clean_text(text):
    if len(text) < 1000000:
        doc = nlplg(text)
        tokens = [token.lower_ for token in doc if token.ent_type_ not in ["PERSON", "CARDINAL", "GPE"] and 
                  token.is_alpha == True and len(token.text) > 1]
        return " ".join(tokens)
    if len(text) >= 1000000:
        doc = textacy.spacier.utils.make_doc_from_text_chunks(text, nlplg)
        tokens = [token.lower_ for token in doc if token.ent_type_ not in ["PERSON", "CARDINAL", "GPE"] and 
                  token.is_alpha == True and len(token.text) > 1]
        return " ".join(tokens)

In [ ]:
def strip_names_and_numbers(text):
    if len(text) < 1000000:
        doc = nlplg(text)
        tokens = [ent.text for ent in doc.ents if ent.label_ not in ["PERSON", "CARDINAL"]]
        return " ".join(tokens)
    if len(text) >= 1000000:
        doc = textacy.spacier.utils.make_doc_from_text_chunks(text, nlplg)
        tokens = [ent.text for ent in doc.ents if ent.label_ not in ["PERSON", "CARDINAL"]]
        return " ".join(tokens)

In [10]:
def strip_places(text):
    if len(text) < 1000000:
        doc = nlplg(text)
        tokens = [ent.text for ent in doc.ents if ent.label_ != "GPE"]
        places = [ent.text for ent in doc.ents if ent.label_ == "GPE"]
        print(len(places))
        return " ".join(tokens)
    if len(text) >= 1000000:
        doc = textacy.spacier.utils.make_doc_from_text_chunks(text, nlplg)
        tokens = [ent.text for ent in doc.ents if ent.label_ != "GPE"]
        places = [ent.text for ent in doc.ents if ent.label_ == "GPE"]
        print(len(places))
        return " ".join(tokens)

In [12]:
with open(test, 'r') as f:
    text = f.read()
    print(len(text))
    no_places = strip_places(text)
    print(len(no_places))

12943
48
4440


In [ ]:
def lowercase(textlist):
    return [word.lower() for word in textlist]

In [ ]:
def remove_numbers(text):
    cleaned = re.sub(r'\d+', '', text)
    return cleaned

In [ ]:
def remove_non_alpha_nltk(text):
    words = word_tokenize(text)
    lowered = lowercase(words)
    return " ".join([word for word in lowered if word.isalpha()])

In [ ]:
def remove_single_letter_words(text):
    cleaned = ' '.join([word for word in text.split() if len(word) > 1])
    return cleaned

In [ ]:
def clean_text(text):
    de_person_number = strip_names_and_numbers(text)
    fewer_nums = remove_numbers(de_person_number)
    alpha = remove_non_alpha_nltk(fewer_nums)
    cleaned = remove_single_letter_words(alpha)
    return cleaned

In [45]:
with open(test, 'r') as f:
    text = f.read()
    cleaned = clean_text(text)
    print(cleaned)

the white hat imniiiniimium unmm arrowsmith and síxpe ny seríesarrow series men in boat to say nothing of the dog the end of life yol diary of pilgrimage iii recalled to life iy frenchman in america and the supernatural the white hat vol vii quay street slmpkin hamilton kent cracker over the copyright entered at stationers hall the white hat and other stories by author of sporting recollections flowers of the hunt etc juustrations bg tbe îlutbor ii quay street london simpkin kent limitednotice the following stories viz the vicissitudes of how miss went to the little fairy and got at have already appeared in the columns of the illustrated sporting and dramatic news to the courtesy of whose proprietor the author is indebted for per mission to republish them contents rage the white hat vicissitudes of hack how miss brown went to the derby little fairy house in the country got at the queen of the arena uncle new way to pay old debts the black sheep of the family game to the last the bishop

In [47]:
len(cleaned)

305317

In [46]:
# Given a filename, extract just the book id
def get_book_id(fn):
    return os.path.split(fn)[1].split('_')[0].split('.')[0]

In [48]:
def write_clean_text(book_id):
    with open(os.path.join('plain', book_id + '.txt'), 'r', encoding='latin-1') as f:
        with open(os.path.join('plain_full_clean', book_id + '.txt'), 'w') as g:
            text = f.read()
            g.write(clean_text(text))

In [49]:
book_ids = {get_book_id(fn) for fn in glob.glob(os.path.join('plain', '*.txt'))}

In [50]:
len(book_ids)

1417

In [59]:
#Since part are writen, if they're written remove the book id from the set of book_ids
written = [get_book_id(fn) for fn in glob.glob("plain_full_clean/*.txt")]

In [61]:
for id in written:
    book_ids.remove(id)

In [62]:
len(book_ids)

401

In [51]:
import multiprocessing as mp

In [63]:
pool = mp.Pool(16)

In [ ]:
mp.cpu_count()

In [64]:
%time pool.map(write_clean_text, book_ids)

CPU times: user 281 ms, sys: 125 ms, total: 406 ms
Wall time: 31min 54s


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [65]:
pool.close()

In [ ]:
for fn in tqdm(glob.glob(os.path.join('plain', '*.txt'))):
    book_id = get_book_id(fn)
    with open(fn, 'r') as f:
        with open(os.path.join('tm_texts', book_id + '.txt'), 'w') as g:
            text = f.read()
            g.write(clean_text(text))